In [1]:


import re
import http.client
import time

conn = http.client.HTTPSConnection("www.ptt.cc")
payload = ''
headers = {
  'cookie': 'over18=1; __cf_bm=lYwo.dvfpkTzgrp8OTgNUuxCDiAvDUtpTTWAzSe6j8U-1636902959-0-Af7hIyc6GOO59bPZsJ/KVKqUDEJsi7i4XRNv6r/VnlXB1WCfLvPsvQPWrHwSN2G2KGr5oExJ6C7RSKZTB7eQS70='
}
conn.request("GET", "/bbs/Beauty/index.html?=", payload, headers)
res = conn.getresponse()
# print(data.decode("utf-8"))

lastBulletin = int(re.search(r"<a class=\"btn wide\" href=\"/bbs/Beauty/index(\d+)\.html\">&lsaquo; 上頁</a>",res.read().decode("utf-8")).group(1))+1

In [2]:

def fetchPage(paath):
    conn.request("GET", paath, payload, headers)
    res = conn.getresponse()
    raw = res.read().decode("utf-8")
    
    # print("push count :",raw.count('class="hl push-tag">推'))
    push_cnt = raw.count('class="hl push-tag">推')
    # print("pull count :",raw.count('class="f1 hl push-tag">噓'))
    pull_cnt = raw.count('class="f1 hl push-tag">噓')

    # print("author:",re.search(r"<span class=\"article-meta-tag\">作者<\/span><span class=\"article-meta-value\">(.*)<\/span><\/div><div\sclass=\"article-metaline-right\"", raw).group(1))
    author = re.search(r"<span class=\"article-meta-tag\">作者<\/span><span class=\"article-meta-value\">(.*)<\/span><\/div><div\sclass=\"article-metaline-right\"", raw).group(1)
    # print("title :",re.search(r"<meta\sproperty=\"og:title\"\scontent=\"(.*)\">",raw).group(1) )
    title = re.search(r"<meta\sproperty=\"og:title\"\scontent=\"(.*)\">",raw).group(1) 
    # print("post time :",re.search(r"<span class=\"article-meta-tag\">時間<\/span><span class=\"article-meta-value\">(.*)<\/span><\/div>",raw).group(1) )
    post_time = re.search(r"<span class=\"article-meta-tag\">時間<\/span><span class=\"article-meta-value\">(.*)<\/span><\/div>",raw).group(1) 
    # print("IP :",re.search(r"<span class=\"f2\">※ 發信站: 批踢踢實業坊\(ptt\.cc\),\s+來自:\s+(\d+\.\d+\.\d+\.\d+)\s+\(.*\)",raw).group(1) )
    IP = re.search(r"<span class=\"f2\">※ 發信站: 批踢踢實業坊\(ptt\.cc\),\s+來自:\s+(\d+\.\d+\.\d+\.\d+)\s+\(.*\)",raw).group(1)
    # print("IMG :",re.search(r"<div class=\"richcontent\"><img src=\"(.*)\" alt=\"\" loading=\"lazy\" />",raw).group(1) )
    try:
        IMG = re.search(r"<div class=\"richcontent\"><img src=\"(.*)\" alt=\"\" loading=\"lazy\" />",raw).group(1)
    except:
        IMG = "NO Image"
    # print(str(time.ctime()))

    # print(author,title,post_time,IMG,IP,push_cnt,pull_cnt,time.ctime())
    return (str(author).strip()+","+str(title).strip()+","+str(post_time).strip()+","
    +str(IMG).strip()+","+str(IP).strip()+","+str(push_cnt)+','+str(pull_cnt).strip()+","+str(time.ctime()) )+"\n"



# fetchPage("/bbs/Beauty/M.1636887390.A.D04.html")


In [3]:
yesterday ={"month" : time.localtime(time.time() - 24*60*60).tm_mon,
        "day":time.localtime(time.time() - 24*60*60).tm_mday}
today = {"month" : time.localtime(time.time() - 0).tm_mon,
        "day":time.localtime(time.time() - 0).tm_mday
}

def DateCmp(M,D):

    if(int(M) == int(yesterday["month"])  and int(D)==int(yesterday["day"]) ):
        return "yesterday"
    elif(int(M) == int(today["month"])  and int(D)==int(today["day"]) ):
        return "today"
    return "out"


keepFetchBulltin = True


In [4]:
output ="作者,標題,時間,第一張圖片網址,po文IP,推文總數,噓文總數,上爬蟲爬取之時間\n"

while (keepFetchBulltin):
    # print(lastBulletin)
    conn.request("GET", "/bbs/Beauty/index"+str(lastBulletin)+".html?=", payload, headers)
    raw = conn.getresponse().read().decode("utf-8")
    raw = raw[:raw.find("r-list-sep")]

    regex = r"<div.*class=\"r-ent\">(\n.*?)*<div.*class=\"mark\">.*<\/div>(\n.*?)*<\/div>(\n.*?)*<\/div>"

    matches = re.finditer(regex, raw, re.MULTILINE)
    for matchNum, match in enumerate(matches, start=1):
        # print(match.group())
        articleDate = re.search(r"<div class=\"date\">(\d+)/(\d+)</div>",match.group())
        # print("Date :", articleDate.group(1),"/",articleDate.group(2))
        # print("title :",re.search(r"<a\s+href=\"(.*)\">(.*)</a>",match.group()).group(0))

        if(DateCmp(articleDate.group(1),articleDate.group(2)) == "yesterday"):
            output += fetchPage(re.search(r"<a\s+href=\"(.*)\">(.*)</a>",match.group()).group(1))
        
        if(DateCmp(articleDate.group(1),articleDate.group(2)) == "out"):
            keepFetchBulltin = False

    lastBulletin -=1






In [5]:
print(output)
with open("ptt_beauty_big5.csv","w",encoding='Big5') as file:
    file.write(output)

with open("ptt_beauty_utf-8.csv","w",encoding='utf-8') as file:
    file.write(output)

作者,標題,時間,第一張圖片網址,po文IP,推文總數,噓文總數,上爬蟲爬取之時間
HarunaOno (Scandal最高),[正妹] 47都道府縣代表女演員,Sun Nov 14 00:05:23 2021,https://cache.ptt.cc/c/https/i.imgur.com/5gFbWCsl.jpg?e=1637036591&amp;s=3giCqnvu3o4bHQ5AbG4PcQ,42.77.25.22,53,4,Mon Nov 15 03:52:02 2021
makilaifan (TS1989),[正妹] 篠崎泫,Sun Nov 14 00:06:52 2021,https://cache.ptt.cc/c/https/i.imgur.com/7Sl8n7Ql.jpg?e=1637077528&amp;s=ESiyNZGPBWwW9ROVY_Po1A,106.107.219.99,70,6,Mon Nov 15 03:52:02 2021
apexi6031 (),[正妹] 左還是右?,Sun Nov 14 00:42:01 2021,https://cache.ptt.cc/c/https/i.imgur.com/H1QUmQ8l.jpg?e=1637023821&amp;s=UVw_yEEe0iT_P0Hptaz45A,114.42.190.87,5,6,Mon Nov 15 03:52:02 2021
playerunknow (SuA),[正妹] 想去圖書館惹,Sun Nov 14 13:47:14 2021,https://cache.ptt.cc/c/https/i.imgur.com/Rms34mgl.jpg?e=1637087709&amp;s=aqsnHBuBiFlawZb33MelLw,1.170.217.118,40,5,Mon Nov 15 03:52:03 2021
showwin (世界一か? 死か?),[神人] 神這位+9,Sun Nov 14 15:03:58 2021,NO Image,114.25.104.224,2,0,Mon Nov 15 03:52:03 2021
ckpot (ckpot_plussizetw),[正妹] 大尺碼｜肉特(A120),Sun Nov 14 17:52:10 2021,